# **[Chapter 02]**  
**┗ 당뇨 환자 병원 재방문 예측**
---
> **목차(Context)**

* 문제상황 및 데이터 살펴보기
* 문제해결 프로세스 정의
* 🥉Session 1 - 「Data 전처리 및 EDA」
* 🥈Session 2 - 「Error analysis」
* 🥇Session 3 - 「주제그룹 분석」

In [16]:
# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

# ▶ Google drive mount or 폴더 클릭 후 구글드라이브 연결
# from google.colab import drive
# drive.mount('/content/drive')

# ▶ 경로 설정 (※ Colab을 활성화시켰다면 보통 Colab Notebooks 폴더가 자동 생성)
import os
os.chdir("/content/drive/MyDrive/인공지능/ML_Project_Collection/당뇨환재재입원예측")
os.getcwd()



'/content/drive/MyDrive/인공지능/ML_Project_Collection/당뇨환재재입원예측'

## **문제상황 및 데이터 살펴보기**
---
> **시나리오** 🏭

```
당뇨병은 세계 성인 인구의 9% 이상에 영향을 미치고 있으며, 점점 더 증가하고 있다.
당뇨 환자는 지속적인 관찰과 치료가 필요하며, 중증 환자는 더 세심한 관리가 필요하다.
일단, 우리의 목적은 30일 이내에 재입원해야 하는 환자를 예측하는 모델을 생성하는 것이다.
병원에서 중증 환자의 모니터링, 조기 예약, 간호사 호출과 같은 예방 조치를 더 많이 취할 수 있다고 생각해보자.
이는, 많은 생명을 구할 수 있는 일이 될지도 모른다.
우리는 당뇨 환자의 재입원을 예측하는 모델을 만들어보고, 모델의 성능 향상을 위해 에러의 분포를 분석하여 성능을 향상시켜보자.
```



In [17]:
# openml API를 사용하여 당뇨 환자 데이터 읽어오기
from sklearn.datasets import fetch_openml
X_orig, y = fetch_openml(data_id=43874, as_frame=True, return_X_y=True)

In [18]:
X_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   race                      101766 non-null  object  
 1   gender                    101766 non-null  object  
 2   age                       101766 non-null  object  
 3   discharge_disposition_id  101766 non-null  object  
 4   admission_source_id       101766 non-null  object  
 5   time_in_hospital          101766 non-null  int64   
 6   medical_specialty         101766 non-null  object  
 7   num_lab_procedures        101766 non-null  int64   
 8   num_procedures            101766 non-null  int64   
 9   num_medications           101766 non-null  int64   
 10  primary_diagnosis         101766 non-null  object  
 11  number_diagnoses          101766 non-null  int64   
 12  max_glu_serum             101766 non-null  object  
 13  A1Cresult                 101

> **데이터 살펴보기**

* 환자 입원 당시 측정 데이터
* 데이터 명세 ⬇

|Column|Description|
|:---|:---|
|Features|환자 입원 기간 내 관련 정보|
|Class|readmit_30_days|

In [19]:
# 각 컬럼별로 값을 실제로 확인해본다.
X_orig.sample(n=5)

,race,gender,age,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,primary_diagnosis,number_diagnoses,max_glu_serum,A1Cresult,insulin,change,diabetesMed,medicare,medicaid,had_emergency,had_inpatient_days,had_outpatient_days,readmitted,readmit_binary
34877,AfricanAmerican,Male,Over 60 years,Discharged to Home,Other,2,Family/GeneralPractice,27,0,15,Other,4,None,None,Steady,Ch,Yes,False,False,False,False,False,NO,0
31560,Caucasian,Female,Over 60 years,Discharged to Home,Emergency,4,Other,45,1,24,Respiratory Issues,4,None,None,No,No,No,False,False,True,False,False,>30,1
66168,Caucasian,Female,Over 60 years,Other,Referral,2,Missing,55,0,13,Diabetes,9,None,None,No,No,No,True,False,False,True,True,NO,0
58490,Caucasian,Female,30-60 years,Discharged to Home,Emergency,2,Emergency/Trauma,66,2,9,Genitourinary Issues,5,None,Norm,Steady,No,Yes,False,False,False,True,False,NO,0
100701,AfricanAmerican,Male,Over 60 years,Discharged to Home,Emergency,8,Missing,55,0,13,Other,9,None,None,No,No,No,False,False,False,False,False,NO,0



## **문제해결 프로세스 정의**
---
> **문제정의**

```
▶ 퇴원한 환자들 중에 다시 입원하는 환자들이 늘어남
```  

> **기대효과**

```
▶ 재입원 환자 예측을 통해 중증 환자 사전 조치 및 모니터링
▶ 중증 환자 생존율 증가
```

> **해결방안**

```
▶ Binary classification을 통해 30일 이내 재입원할 환자를 분류
▶ Session 1 🥉
 - 인코딩을 통한 데이터 준비
 - 베이스 모델 생성하기
▶ Session 2 🥈
 - Class weight가 오류에 미치는 영향 분석
 - 하이퍼파라미터 튜닝
▶ Session 3 🥇
 - 에러를 서로 다른 주제 그룹으로 나눠보기
 - Feature importance 알아보기

기타: 에러를 줄일 수 있는 몇가지 방법에 대한 이야기
```

> **성과측정**  

```
▶ 에러 분석을 통한 점진적 성능 향상
```

> **현업적용**  

```
▶ 모델 성능 하락시 분석 포인트 탐색
```

> **주요 핵심 미리 살펴보기**  

```
▶ Session 1 → OneHotEncoder를 활용한 Feature 생성
▶ Session 2 → LightGBMClassifier를 활용한 베이스 모델 생성, optuna를 활용한 Hyperparameter 튜닝
▶ Session 3 → 일반적인 에러 분석, Cohort 에러 분석
```

## **Data Spec Check**
---
> **Data 가공 명세서**

<img src = "https://drive.google.com/uc?id=10ghjKrqQWBHS4dYzol3IIU1CzJ_vv4Ih">

## **🥉Session 1**  
**┗ Data 전처리 및 EDA**  
---

### 1-0. 몇 가지 필요한 Library 설치

In [20]:
!pip install raiwidgets lightgbm optuna shap --quiet

### 1-1. 필요한 패키지 Import

In [21]:
import raiwidgets
print(dir(raiwidgets))


['ErrorAnalysisDashboard', 'ExplanationDashboard', 'FairnessDashboard', 'ModelAnalysisDashboard', 'ModelPerformanceDashboard', 'ResponsibleAIDashboard', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'constants', 'dashboard', 'error_analysis_constants', 'error_analysis_dashboard', 'error_analysis_dashboard_input', 'error_handling', 'explanation_constants', 'explanation_dashboard', 'explanation_dashboard_input', 'fairness_dashboard', 'fairness_metric_calculation', 'interfaces', 'model_analysis_dashboard', 'model_performance_dashboard', 'responsibleai_dashboard', 'responsibleai_dashboard_input', 'version']


In [22]:
import math
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',50)


from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

import lightgbm as lgb
import optuna

from responsibleai import RAIInsights
from raiwidgets import ResponsibleAIDashboard
# from raiwidgets.cohort import Cohort, CohortFilter, CohortFilterMethods
import shap

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import seaborn as sns